#### **데이터 불러오기**

In [ ]:
# 메서드 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import time

# Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
# from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier

# Voting
from sklearn.ensemble import VotingClassifier


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Metrics
from sklearn.metrics import f1_score, confusion_matrix

In [ ]:
plt.rc('font', family='NanumBarunGothic')

In [ ]:
# 코랩 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ROOT = '/content/drive/MyDrive/2022빅콘/data/'

In [ ]:
# 데이터 불러오기
train = pd.read_csv(ROOT + 'Final/train_under.csv')
valid = pd.read_csv(ROOT + 'Final/valid.csv')

In [ ]:
train.drop(['신청서 번호','금융사 번호','상품 번호','유저 번호','생성일시'], axis=1, inplace=True)
valid.drop(['신청서 번호','금융사 번호','상품 번호','유저 번호','생성일시'], axis=1, inplace=True)

In [ ]:
train = pd.get_dummies(data=train, columns=['주거소유형태','대출목적','연령대'])
valid = pd.get_dummies(data=valid, columns=['주거소유형태','대출목적','연령대'])

In [ ]:
train_num = train[['추천 수','승인금리','승인한도','근속월수(로그)','연소득(boxcox)','신용점수(보완)','시간차(로그)','대출희망금액(boxcox)','기대출수(로그)','상품 누적 추천 수(로그)','조회 횟수(로그)']]
valid_num = valid[['추천 수','승인금리','승인한도','근속월수(로그)','연소득(boxcox)','신용점수(보완)','시간차(로그)','대출희망금액(boxcox)','기대출수(로그)','상품 누적 추천 수(로그)','조회 횟수(로그)']] 

In [ ]:
# MinMaxScaler
Scaler = MinMaxScaler()

train_num_mm = Scaler.fit_transform(train_num)
valid_num_mm = Scaler.transform(valid_num)

train_num_mm = pd.DataFrame(train_num_mm, columns=['추천 수','승인금리','승인한도','근속월수(로그)','연소득(boxcox)','신용점수(보완)','시간차(로그)','대출희망금액(boxcox)','기대출수(로그)','상품 누적 추천 수(로그)','조회 횟수(로그)'])
valid_num_mm = pd.DataFrame(valid_num_mm, columns=['추천 수','승인금리','승인한도','근속월수(로그)','연소득(boxcox)','신용점수(보완)','시간차(로그)','대출희망금액(boxcox)','기대출수(로그)','상품 누적 추천 수(로그)','조회 횟수(로그)'])

In [ ]:
train.drop(['추천 수','승인금리','승인한도','근속월수(로그)','연소득(boxcox)','신용점수(보완)','시간차(로그)','대출희망금액(boxcox)','기대출수(로그)','상품 누적 추천 수(로그)','조회 횟수(로그)'], axis=1, inplace=True)
valid.drop(['추천 수','승인금리','승인한도','근속월수(로그)','연소득(boxcox)','신용점수(보완)','시간차(로그)','대출희망금액(boxcox)','기대출수(로그)','상품 누적 추천 수(로그)','조회 횟수(로그)'], axis=1, inplace=True)

train = pd.concat([train, train_num_mm], axis=1)
valid = pd.concat([valid, valid_num_mm], axis=1)

In [ ]:
X_train = train.drop(['신청 여부(타겟)'], axis=1)
y_train = train['신청 여부(타겟)']

X_valid = valid.drop(['신청 여부(타겟)'], axis=1)
y_valid = valid['신청 여부(타겟)']

 min_impurity_split=None,

In [ ]:
# Voting Classifier
voting_c =  VotingClassifier(estimators=[('rf',RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                                                                      criterion='gini', max_depth=None, max_features='auto',
                                                                      max_leaf_nodes=None, max_samples=None,
                                                                      min_impurity_decrease=0.0,
                                                                      min_samples_leaf=1, min_samples_split=2,
                                                                      min_weight_fraction_leaf=0.0, n_estimators=100,
                                                                      n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                                                                      warm_start=False)),
                                         ('lgbm',LGBMClassifier(bagging_fraction=0.7283211870996462, bagging_freq=5,
                                                                boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                                                                feature_fraction=0.8601639377763195, importance_type='split',
                                                                learning_rate=0.09314704228715577, max_depth=-1,
                                                                min_child_samples=91, min_child_weight=0.001,
                                                                min_split_gain=0.44317785409769705, n_estimators=70, n_jobs=-1,
                                                                num_leaves=239, objective=None, random_state=42,
                                                                reg_alpha=1.329342606494723, reg_lambda=0.0012121985154727188,
                                                                silent='warn', subsample=1.0, subsample_for_bin=200000,
                                                                subsample_freq=0)),
                                         ('et',ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                                                                    criterion='gini', max_depth=None, max_features='auto',
                                                                    max_leaf_nodes=None, max_samples=None,
                                                                    min_impurity_decrease=0.0,
                                                                    min_samples_leaf=1, min_samples_split=2,
                                                                    min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                                                                    oob_score=False, random_state=42, verbose=0,
                                                                    warm_start=False))],
                             voting='soft')

In [ ]:
# Voting Classifier
voting_h =  VotingClassifier(estimators=[('rf',RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                                                                      criterion='gini', max_depth=None, max_features='auto',
                                                                      max_leaf_nodes=None, max_samples=None,
                                                                      min_impurity_decrease=0.0,
                                                                      min_samples_leaf=1, min_samples_split=2,
                                                                      min_weight_fraction_leaf=0.0, n_estimators=100,
                                                                      n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                                                                      warm_start=False)),
                                         ('lgbm',LGBMClassifier(bagging_fraction=0.7283211870996462, bagging_freq=5,
                                                                boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                                                                feature_fraction=0.8601639377763195, importance_type='split',
                                                                learning_rate=0.09314704228715577, max_depth=-1,
                                                                min_child_samples=91, min_child_weight=0.001,
                                                                min_split_gain=0.44317785409769705, n_estimators=70, n_jobs=-1,
                                                                num_leaves=239, objective=None, random_state=42,
                                                                reg_alpha=1.329342606494723, reg_lambda=0.0012121985154727188,
                                                                silent='warn', subsample=1.0, subsample_for_bin=200000,
                                                                subsample_freq=0)),
                                         ('et',ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                                                                    criterion='gini', max_depth=None, max_features='auto',
                                                                    max_leaf_nodes=None, max_samples=None,
                                                                    min_impurity_decrease=0.0,
                                                                    min_samples_leaf=1, min_samples_split=2,
                                                                    min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                                                                    oob_score=False, random_state=42, verbose=0,
                                                                    warm_start=False))],
                             voting='hard')

In [ ]:
voting_c.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(n_jobs=-1,
                                                     random_state=42)),
                             ('lgbm',
                              LGBMClassifier(bagging_fraction=0.7283211870996462,
                                             bagging_freq=5,
                                             feature_fraction=0.8601639377763195,
                                             learning_rate=0.09314704228715577,
                                             min_child_samples=91,
                                             min_split_gain=0.44317785409769705,
                                             n_estimators=70, num_leaves=239,
                                             random_state=42,
                                             reg_alpha=1.329342606494723,
                                             reg_lambda=0.0012121985154727188,
                                            

In [ ]:
y_pred = voting_c.predict(X_valid)

In [ ]:
y_pred

array([1., 0., 0., ..., 0., 0., 0.])

In [ ]:
f1_score(y_valid, y_pred)

0.3346596807217658

In [ ]:
confusion_matrix(y_valid, y_pred)

array([[1955671,  471747],
       [  16008,  122668]])

In [ ]:
data = pd.read_csv(ROOT + 'Dataset.csv')

In [ ]:
submit = pd.read_csv(ROOT + '데이터분석분야_퓨처스부문_평가데이터11.csv')

In [ ]:
submit['신청 여부(타겟)'] = 0

In [ ]:
submit.columns = ['신청서 번호','상품 번호','신청 여부(타겟)']

In [ ]:
test = data[data['신청 여부(타겟)'].isna()]

In [ ]:
test.isna().sum()

신청서 번호                  0
금융사 번호                  0
상품 번호                   0
신청 여부(타겟)         3255466
유저 번호                   0
생성일시                    0
추천 수                    0
승인금리                    0
승인한도                    0
근속월수(로그)                0
연소득(boxcox)             0
신용점수(보완)                0
시간차(로그)                 0
대출희망금액(boxcox)          0
기대출수(로그)                0
상품 누적 추천 수(로그)          0
조회 횟수(로그)               0
주거소유형태_기타가족소유           0
주거소유형태_자가               0
주거소유형태_전월세              0
대출목적_기타                 0
대출목적_대환대출               0
대출목적_사업자금               0
대출목적_생활비                0
대출목적_자동차구입              0
대출목적_전월세보증금             0
대출목적_주택구입               0
대출목적_투자                 0
연령대_20대                 0
연령대_30대                 0
연령대_40대                 0
연령대_50대                 0
연령대_60대                 0
연령대_70대                 0
연령대_80대 이상              0
연령대_알수없음                0
dtype: int64

In [ ]:
test = pd.get_dummies(data=test, columns=['주거소유형태','대출목적','연령대'])

In [ ]:
test_num = test[['추천 수','승인금리','승인한도','근속월수(로그)','연소득(boxcox)','신용점수(보완)','시간차(로그)','대출희망금액(boxcox)','기대출수(로그)','상품 누적 추천 수(로그)','조회 횟수(로그)']] 

In [ ]:
test_num_mm = Scaler.transform(test_num)
test_num_mm = pd.DataFrame(test_num_mm, columns=['추천 수','승인금리','승인한도','근속월수(로그)','연소득(boxcox)','신용점수(보완)','시간차(로그)','대출희망금액(boxcox)','기대출수(로그)','상품 누적 추천 수(로그)','조회 횟수(로그)'])

In [ ]:
test.drop(['추천 수','승인금리','승인한도','근속월수(로그)','연소득(boxcox)','신용점수(보완)','시간차(로그)','대출희망금액(boxcox)','기대출수(로그)','상품 누적 추천 수(로그)','조회 횟수(로그)'], axis=1, inplace=True)

In [ ]:
test.columns

Index(['신청서 번호', '금융사 번호', '상품 번호', '신청 여부(타겟)', '유저 번호', '생성일시',
       '주거소유형태_기타가족소유', '주거소유형태_자가', '주거소유형태_전월세', '대출목적_기타', '대출목적_대환대출',
       '대출목적_사업자금', '대출목적_생활비', '대출목적_자동차구입', '대출목적_전월세보증금', '대출목적_주택구입',
       '대출목적_투자', '연령대_20대', '연령대_30대', '연령대_40대', '연령대_50대', '연령대_60대',
       '연령대_70대', '연령대_80대 이상', '연령대_알수없음'],
      dtype='object')

In [ ]:
test.reset_index(drop=True, inplace=True)

In [ ]:
test = pd.concat([test, test_num_mm], axis=1)

In [ ]:
submit_test = test[['신청서 번호','상품 번호','신청 여부(타겟)']]

In [ ]:
submit_test

,신청서 번호,상품 번호,신청 여부(타겟)
0,1748340,191,NaN
1,1748340,169,NaN
2,1748340,7,NaN
3,1748340,268,NaN
4,1748340,118,NaN
...,...,...,...
3255461,1428218,200,NaN
3255462,1428218,7,NaN
3255463,1428218,257,NaN
3255464,1428218,110,NaN


In [ ]:
X_test = test.drop(['신청 여부(타겟)','신청서 번호','금융사 번호','상품 번호','유저 번호','생성일시'], axis=1)
y_test = test['신청 여부(타겟)']

In [ ]:
# 최종 예측
y_pred = voting_c.predict(X_test)

In [ ]:
y_pred

array([0., 0., 0., ..., 0., 0., 1.])

In [ ]:
submit_test['신청 여부(타겟)'] = y_pred

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
submit_test

,신청서 번호,상품 번호,신청 여부(타겟)
0,1748340,191,0.0
1,1748340,169,0.0
2,1748340,7,0.0
3,1748340,268,1.0
4,1748340,118,0.0
...,...,...,...
3255461,1428218,200,0.0
3255462,1428218,7,1.0
3255463,1428218,257,0.0
3255464,1428218,110,0.0


In [ ]:
submit

,신청서 번호,상품 번호,신청 여부(타겟)
0,4,220,0
1,4,191,0
2,8,29,0
3,8,159,0
4,8,85,0
...,...,...,...
1048570,697978,205,0
1048571,697980,206,0
1048572,697980,140,0
1048573,697980,263,0


In [ ]:
real_submit = pd.merge(submit, submit_test, on=['신청서 번호','상품 번호']).drop(['신청 여부(타겟)_x'], axis=1)
real_submit.columns = ['신청서 번호','상품 번호','신청 여부(타겟)']
real_submit

,신청서 번호,상품 번호,신청 여부(타겟)
0,4,220,1.0
1,4,191,1.0
2,8,29,1.0
3,8,159,1.0
4,8,85,1.0
...,...,...,...
1048570,697978,205,1.0
1048571,697980,206,0.0
1048572,697980,140,1.0
1048573,697980,263,0.0


In [ ]:
real_submit['신청 여부(타겟)'].value_counts()

0.0    806235
1.0    242340
Name: 신청 여부(타겟), dtype: int64

In [ ]:
real_submit.to_csv(ROOT + '데이터분석분야_퓨처스부문_인생핀다_평가데이터.csv', index=False)